In [54]:
from dotenv import load_dotenv
load_dotenv()


True

In [21]:
# Phoenix can display in real time the traces automatically
# collected from your LlamaIndex application.
import phoenix as px

# Look for a URL in the output to open the App in a browser.
px.launch_app()
# The App is initially empty, but as you proceed with the steps below,
# traces will appear automatically as your LlamaIndex application runs.

import llama_index

llama_index.set_global_handler("arize_phoenix")

# Run all of your LlamaIndex applications as usual and traces
# will be collected and displayed in Phoenix.

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://127.0.0.1:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [10]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
# Option: if developing with the llama_hub package
# from llama_hub.llama_packs.agents.llm_compiler.step import LLMCompilerAgentWorker

# Option: download_llama_pack
from llama_index.llama_pack import download_llama_pack

download_llama_pack(
    "LLMCompilerAgentPack",
    "./agent_pack",
    skip_load=True,
    # leave the below line commented out if using the notebook on main
    # llama_hub_url="https://raw.githubusercontent.com/run-llama/llama-hub/jerry/add_llm_compiler_pack/llama_hub"
)
from agent_pack.step import LLMCompilerAgentWorker

In [12]:
import json
from typing import Sequence, List

from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [13]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

tools = [multiply_tool, add_tool]

In [14]:
multiply_tool.metadata.fn_schema_str

"{'title': 'multiply', 'type': 'object', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b']}"

In [15]:
from llama_index.agent import AgentRunner

In [55]:
llm = OpenAI(model="gpt-4", api_key=os.getenv("OPENAI_API_KEY"))

In [38]:
callback_manager = llm.callback_manager

In [39]:
callback_manager

In [40]:
agent_worker = LLMCompilerAgentWorker.from_tools(
    tools, llm=llm, verbose=True, callback_manager=callback_manager
)
agent = AgentRunner(agent_worker, callback_manager=callback_manager)

In [41]:
response = agent.chat("What is (121 * 3) + 42?")

> Running step 2efa3d5b-6490-4a04-8131-f6814ec391db for task 71217ffd-9655-4beb-a31b-0bec47327cc5.
> Step count: 0
> Plan: 1. multiply(121, 3)
2. add($1, 42)
3. join()<END_OF_PLAN>
Ran task: multiply. Observation: 363
Ran task: add. Observation: 405
Ran task: join. Observation: None
> Thought: The result of the operation is 405.
> Answer: 405


In [42]:
response

AgentChatResponse(response='405', sources=[], source_nodes=[])

In [43]:
agent.memory.get_all()

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is (121 * 3) + 42?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='405', additional_kwargs={})]

In [47]:
from llama_index.readers import WikipediaReader

In [45]:
wiki_titles = ["Toronto", "Seattle", "Chicago", "Boston", "Miami"]

In [50]:
city_docs = {}
reader = WikipediaReader()
for wiki_title in wiki_titles:
    docs = reader.load_data(pages=[wiki_title])
    city_docs[wiki_title] = docs

In [56]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import CallbackManager

llm = OpenAI(temperature=0, model="gpt-4", api_key=os.getenv("OPENAI_API_KEY"))
service_context = ServiceContext.from_defaults(llm=llm)
callback_manager = CallbackManager([])

In [57]:
from llama_index import load_index_from_storage, StorageContext
from llama_index.node_parser import SentenceSplitter
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index import VectorStoreIndex
import os

node_parser = SentenceSplitter()

# Build agents dictionary
query_engine_tools = []

for idx, wiki_title in enumerate(wiki_titles):
    nodes = node_parser.get_nodes_from_documents(city_docs[wiki_title])

    if not os.path.exists(f"./data/{wiki_title}"):
        # build vector index
        vector_index = VectorStoreIndex(
            nodes, service_context=service_context, callback_manager=callback_manager
        )
        vector_index.storage_context.persist(persist_dir=f"./data/{wiki_title}")
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{wiki_title}"),
            service_context=service_context,
            callback_manager=callback_manager,
        )
    # define query engines
    vector_query_engine = vector_index.as_query_engine()

    # define tools
    query_engine_tools.append(
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name=f"vector_tool_{wiki_title}",
                description=(
                    "Useful for questions related to specific aspects of"
                    f" {wiki_title} (e.g. the history, arts and culture,"
                    " sports, demographics, or more)."
                ),
            ),
        )
    )

In [58]:
from llama_index.agent import AgentRunner
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-4")
agent_worker = LLMCompilerAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    callback_manager=callback_manager,
)
agent = AgentRunner(agent_worker, callback_manager=callback_manager)

In [59]:
response = agent.chat(
    "Tell me about the demographics of Miami, and compare that with the demographics of Chicago?"
)
print(str(response))

> Running step 5ab3a6b8-da1e-455f-93a1-046e0a3200ae for task 8f067534-0333-4944-a269-5cf81e64c6c1.
> Step count: 0
> Plan: 1. vector_tool_Miami('demographics')
2. vector_tool_Chicago('demographics')
3. join()<END_OF_PLAN>
Ran task: vector_tool_Chicago. Observation: Chicago has a diverse demographic makeup. During its first hundred years, it was one of the fastest-growing cities in the world, with its population growing from under 200 in 1833 to over 1 million by 1890. The city saw waves of immigrants from various parts of the world, including Ireland, Southern, Central and Eastern Europe, and Africa. The city's black population doubled between 1910 and 1920 and again between 1920 and 1930. Most of Chicago's foreign-born population were born in Mexico, Poland, and India. As of July 2019, the largest racial or ethnic group in Chicago is non-Hispanic White at 32.8% of the population, followed by Blacks at 30.1% and the Hispanic population at 29.0%. The city also has the third-largest LGBT

### Note: they seem to still run sequentially 

In [60]:
response = agent.chat(
    "Is the climate of Chicago or Seattle better during the wintertime?"
)
print(str(response))

> Running step b601eb45-f61d-42fa-8268-c621c3915eb3 for task 29c5df82-c70a-410a-984d-07c58f3419bf.
> Step count: 0
> Plan: 1. vector_tool_Chicago('climate during wintertime')
2. vector_tool_Seattle('climate during wintertime')
3. join()<END_OF_PLAN>
Ran task: vector_tool_Seattle. Observation: During wintertime, Seattle experiences cool, wet weather. Extreme cold temperatures, below about 15 °F or -9 °C, are rare due to the moderating influence of the adjacent Puget Sound, the greater Pacific Ocean, and Lake Washington. The city is often cloudy due to frequent storms and lows moving in from the Pacific Ocean, leading to many "rain days".
Ran task: vector_tool_Chicago. Observation: During wintertime, the city experiences relatively cold and snowy conditions. Blizzards can occur, as they did in winter 2011. There are many sunny but cold days. The normal winter high from December through March is about 36 °F (2 °C). January and February are the coldest months. A polar vortex in January 201